In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install datasets
!pip install langchain
!pip install -U langchain-community
!pip install chromadb
# !pip install faiss-gpu
import torch
from datasets import load_dataset
from transformers import LlavaForConditionalGeneration, AutoProcessor
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma # FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from PIL import Image
import pandas as pd
import networkx as nx
from tqdm import tqdm
import re

IMAGE_PATH = "/content/drive/MyDrive/MATH-V-main"

class MathVisionEvaluator:
    def __init__(self, dataset_name="MathLLMs/MathVision"):
        """
        Initialize the evaluator with RAG and graphRAG capabilities
        """
        # Load dataset and model as before
        self.dataset = load_dataset(dataset_name)
        self.model_id = "llava-hf/llava-1.5-7b-hf"
        self.fine_tune_path = "/content/drive/MyDrive/reasoning_multimodal_LLMs/outputs"
        self.model = LlavaForConditionalGeneration.from_pretrained(
            self.model_id,
            torch_dtype=torch.float16,
            device_map='auto'
        )

        self.fine_tuned_model = LlavaForConditionalGeneration.from_pretrained(
            self.fine_tune_path,
            torch_dtype=torch.float16,
            device_map='auto'
        )

        self.processor = AutoProcessor.from_pretrained(self.model_id)

        # Initialize RAG components
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )

        # Create knowledge base from training data
        self._initialize_knowledge_base()

        # Create problem graph
        self._initialize_problem_graph()

    def _initialize_knowledge_base(self, method='Chroma'):
        """
        Initialize RAG knowledge base from training data
        """

        # Prepare documents from training data
        documents = []
        metadatas = []
        for example in self.dataset['test']:
            doc = f"Question: {example['question']}\nAnswer: {example['answer']}\nSubject: {example['subject']}\nlevel: {example['level']}"
            documents.append(doc)
            metadatas.append({
                'subject': example['subject'],
                'level': example['level']
            })

        if method=='Chroma': #Chroma
            # Create Chroma vector store
            self.vector_store = Chroma.from_texts(
                texts=documents,
                metadatas=metadatas,
                embedding=self.embeddings,
                persist_directory="./chroma_db"  # This will persist the database locally
            )
        else: # FAISS
            # Split documents into chunks
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=500,
                chunk_overlap=50
            )
            texts = text_splitter.create_documents(documents)

            # Create vector store
            self.vector_store = FAISS.from_documents(texts, self.embeddings)


    def _initialize_problem_graph(self):
        """
        Create a graph representation of problems and their relationships
        """
        self.problem_graph = nx.Graph()

        # Add nodes for each training example
        for i, example in enumerate(self.dataset['test']):
            self.problem_graph.add_node(i,
                                      subject=example['subject'],
                                      level=example['level'],
                                      question=example['question'],
                                      answer=example['answer'])

        # Add edges between similar problems (based on subject and level)
        for i in range(len(self.dataset['test'])):
            for j in range(i + 1, len(self.dataset['test'])):
                if (self.problem_graph.nodes[i]['subject'] == self.problem_graph.nodes[j]['subject'] and
                    abs(self.problem_graph.nodes[i]['level'] - self.problem_graph.nodes[j]['level']) <= 1):
                    self.problem_graph.add_edge(i, j, weight=1)

    def _get_relevant_context(self, question, subject, level, method='rag'):
        """
        Get relevant context using either RAG or graphRAG
        """
        if method == 'rag':
            # Use traditional RAG to find similar problems
            similar_docs = self.vector_store.similarity_search(
                f"Question: {question} level: {level}", k=3, # Subject: {subject}
                filter={"subject": subject}
            )
            context = "\n\n".join([doc.page_content for doc in similar_docs])

        else:  # graph_rag
            # Find most similar node in graph
            similar_problems = []
            for node in self.problem_graph.nodes():
                node_data = self.problem_graph.nodes[node]
                if (node_data['subject'] == subject and
                    abs(node_data['level'] - level) <= 1):
                    similar_problems.append(node)

            # Get connected problems from graph
            context_problems = []
            for problem in similar_problems[:2]:  # Get top 2 similar problems
                neighbors = list(self.problem_graph.neighbors(problem))
                if neighbors:
                    context_problems.extend([self.problem_graph.nodes[n] for n in neighbors[:2]])

            context = "\n\n".join([
                f"Question: {p['question']}\nAnswer: {p['answer']}"
                for p in context_problems
            ])

        return context

    def _prepare_input(self, example, method='zero_shot'):
        """
        Prepare input with optional RAG/graphRAG context
        """
        base_prompt = f"Solve the following math problem step by step, given the image attached. Write the final answer after <Answer> \n{example['question']}"

        if method in ['rag', 'graph_rag']:
            context = self._get_relevant_context(
                example['question'],
                example['subject'],
                example['level'],
                method=method
            )
            base_prompt = f"Here are some similar problems and their solutions:\n{context}\n\nNow solve this problem:\n{base_prompt}"

        conversation = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": base_prompt},
                ],
            },
        ]
        prompt = self.processor.apply_chat_template(conversation, add_generation_prompt=True)

        # Process image if available
        if 'image' in example and example['image']:
            images = [Image.open(f"{IMAGE_PATH}/{example['image']}")]
        else:
            images = None

        inputs = self.processor(
            images=images,
            text=[prompt],
            padding=True,
            return_tensors="pt"
        ).to(self.model.device, torch.float16)

        return inputs

    def evaluate(self, methods=['zero_shot', 'rag', 'graph_rag']):
        """
        Evaluate using specified methods
        """
        for method in methods:
            model_results = []
            for i, example in enumerate(tqdm(self.dataset['testmini'],
                                          desc=f"Evaluating Llava - {method}")):
                inputs = self._prepare_input(example, method=method)

                if method == 'fine_tuned':
                    model = self.fine_tuned_model
                else:
                    model = self.model

                generate_ids = model.generate(
                    **inputs,
                    max_new_tokens=1024,
                )

                generated_text = self.processor.batch_decode(
                    generate_ids,
                    skip_special_tokens=True
                )
                extracted_solution = self._extract_solution(generated_text)

                model_results.append({
                    'question': example['question'],
                    'ground_truth': example['answer'],
                    'model_prediction': extracted_solution,
                    'method': method
                })

                if i % 50 == 0:
                    pd.DataFrame(model_results).to_csv(f'{IMAGE_PATH}/{method}_results.csv')

            self.results[method] = model_results

        return self.results

    def _extract_solution(self, generated_text):
        """Extract solution from generated text"""
        solution_match = re.search(r'<Answer>\s*(.*)', generated_text[0], re.DOTALL)
        if solution_match:
            return solution_match.group(1).strip()
        return generated_text[0]

# Usage
methods = ['zero_shot', 'rag', 'graph_rag'] # , 'fine_tuned'
evaluator = MathVisionEvaluator()
results = evaluator.evaluate(methods) # 'zero_shot', 'rag', 'graph_rag'

# Save results
for method in results:
    pd.DataFrame(results[method]).to_csv(f'{IMAGE_PATH}/{method}_results.csv')

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.35
    Uninstalling langchain-core-0.3.35:
      Successfully uninstalled langchain-core-0.3.35
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.18
    Uninstalling langchain-0.3.18:
      Successfully uninstalled 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.93k [00:00<?, ?B/s]

(…)-00000-of-00001-3532b8d3f1b4047a.parquet:   0%|          | 0.00/57.0M [00:00<?, ?B/s]

(…)-00000-of-00001-f8ff70fcb2f29b1d.parquet:   0%|          | 0.00/6.99M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Evaluating Llava - zero_shot:  58%|█████▊    | 177/304 [25:36<10:58,  5.18s/it]

## Evaluate

In [ ]:
from google.colab import drive
import pandas as pd
import re
drive.mount('/content/drive')
IMAGE_PATH = "/content/drive/MyDrive/MATH-V-main"

def extract_solution(text):
    """Extract solution from generated text"""
    pattern = r'(?:<Answer>|Answer:)\s*[:]*\s*([\w\d]+)\.*'
    solution_match = re.search(pattern, text)
    return solution_match.group(1).strip() if solution_match else None


for method in ['fine_tuned', 'zero_shot', 'rag', 'graph_rag']:
    df = pd.read_csv(f'{IMAGE_PATH}/{method}_results.csv', index_col=0)
    df['solution'] = df.apply(lambda x: extract_solution(x['model_prediction']), axis=1)
    print(f"{method} Accuracy: {(df['ground_truth'].astype(str)==df['solution']).mean()}")

In [ ]:
df[['solution', 'ground_truth']]

In [ ]:
idx = 4
txt = df.loc[idx, 'model_prediction']
print(txt)
print('ground truth = ', df.loc[idx, 'ground_truth'])
pattern = r'(?:<Answer>|Answer:)\s*[:]*\s*([\w\d]+)\.*'
re.search(pattern, txt).group(1).strip()